In [1]:
class Node(object):
    def __init__(self, name):
        """
        :param name: a string
        """
        self.name = name
    def get_name(self):
        return self.name
    def __str__(self):
        return self.name

class Edge(object):
    def __init__(self, src, dest):
        """
        :param src: node
        :param dest: node
        """
        self._src = src
        self._dest = dest
    def get_source(self):
        return self._src
    def get_destination(self):
        return self._dest
    def __str__(self):
        return self._src.get_name() + " -> " + self._dest.get_name()

class Weighted_Edge(Edge):
    def __init__(self, src, dest, weight = 1.0):
        """
        :param src: node
        :param dest: node
        :param weight: a number
        """
        self._src = src
        self._dest = dest
        self._weight = weight
    def get_weight(self):
        return self._weight
    def __str__(self):
        return self._src.get_name() + " -> " + self._dest.get_name() + " weight: " + str(self._weight)

In [2]:
class Digraph(object):
    """
    nodes is a list of the nodes in the graph
    edges is a dict mapping rach node to a list of it's children
    """
    def __init__(self):
        self._nodes = []
        self._edges = {}
    def add_node(self, node):
        if node in self._nodes:
            raise ValueError("Node already exists")
        else:
            self._nodes.append(node)
            self._edges[node] = []
    def add_edge(self, edge):
        src = edge.get_source()
        dest = edge.get_destination()
        if not (src in self._nodes or dest in self._nodes):
            raise ValueError("Node does not exist")
        self._edges[src].append(dest)
    def children_of(self, node):
        return self._edges[node]
    def has_node(self, node):
        return node in self._nodes
    def __str__(self):
        result = ''
        for src in self._nodes:
            for dest in self._edges[src]:
                result = (result + src.get_name() + " -> " + dest.get_name() + '\n')
        return result[:-1] #omit final newline

class Graph(Digraph):
    def add_edge(self, edge):
        Digraph.add_edge(self, edge)
        rev = Edge(edge.get_destination(), edge.get_source())
        Digraph.add_edge(self, rev)

In [6]:
def print_path(path):
    """
    :param path: a list of nodes
    :return: string
    """
    result = ''
    for node in range(len(path)):
        result += str(path[node])
        if node != len(path) - 1:
            result += '->'
    return result

def DFS(graph, start, end, path, shortest, to_print = False):
    """
    :param graph: Digraph
    :param start: node
    :param end: node
    :param path: list of nodes
    :param shortest: shortest list of nodes
    :param to_print: bool
    :return: a shortest path from start to end
    """
    path = path + [start]
    if to_print:
        print("Current DFS path:", print_path(path))
    if start == end:
        return path
    for node in graph.children_of(start):
        if node not in path: #avoid cycles
            if shortest == None or len(path) < len(shortest):
                new_path = DFS(graph, node, end, path, shortest, to_print)
                if new_path != None:
                    shortest = new_path
    return shortest

def shortest_path(graph, start, end, to_print=False):
    """
    :param graph: Digraph
    :param start: node
    :param end: node
    :param to_print: bool
    :return: shortest path from start to end
    """
    return DFS(graph, start, end, [], None, to_print)

The algorithm above is an example of a recursive depth-first-search (DFS) algorithm

In [7]:
def test_SP():
    nodes = []
    for name in range(6): #create 6 nodes
        nodes.append(Node(str(name)))
    g = Digraph()
    for n in nodes:
        g.add_node(n)
    g.add_edge(Edge(nodes[0], nodes[1]))
    g.add_edge(Edge(nodes[1], nodes[2]))
    g.add_edge(Edge(nodes[2], nodes[3]))
    g.add_edge(Edge(nodes[2], nodes[4]))
    g.add_edge(Edge(nodes[3], nodes[4]))
    g.add_edge(Edge(nodes[3], nodes[5]))
    g.add_edge(Edge(nodes[0], nodes[2]))
    g.add_edge(Edge(nodes[1], nodes[0]))
    g.add_edge(Edge(nodes[3], nodes[1]))
    g.add_edge(Edge(nodes[4], nodes[0]))
    sp = shortest_path(g, nodes[0], nodes[5], to_print=True)
    print('Shortest path found by DFS:', print_path(sp))

In [8]:
test_SP()

Current DFS path: 0
Current DFS path: 0->1
Current DFS path: 0->1->2
Current DFS path: 0->1->2->3
Current DFS path: 0->1->2->3->4
Current DFS path: 0->1->2->3->5
Current DFS path: 0->1->2->4
Current DFS path: 0->2
Current DFS path: 0->2->3
Current DFS path: 0->2->3->4
Current DFS path: 0->2->3->5
Current DFS path: 0->2->3->1
Current DFS path: 0->2->4
Shortest path found by DFS: 0->2->3->5


In [ ]:
#Modify the DFS algorith to find a path that minimizes the sum of the weights. Assume that all weights are positive integers